# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

- Pessoa(__Id__, Gender, Age)  
- Medicamento(__Code__, Name)  
- UsoMedicamento(__Pessoa_id__, __Drug_Code__, DaysUse)  
    - CHE - Id from Pessoa  
    - CHE - Code from Medicamento

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Medicamento;
DROP TABLE IF EXISTS UsoMedicamento;

In [3]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [4]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [5]:
CREATE TABLE Medicamento (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(80),
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

In [6]:
SELECT * FROM Medicamento;

In [7]:
CREATE TABLE UsoMedicamento (
  PessoaId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PessoaId)
    REFERENCES Pessoa(Id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Medicamento(Code)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [8]:
SELECT * FROM UsoMedicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [9]:
SELECT * FROM Pessoa
    WHERE Age > 60
    ORDER BY AGE;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [10]:
SELECT DISTINCT Age FROM Pessoa;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [11]:
SELECT Pessoa.Id, Pessoa.Age, UsoMedicamento.DrugCode, UsoMedicamento.DaysUse
    FROM Pessoa, UsoMedicamento
    WHERE Pessoa.Id = UsoMedicamento.PessoaId;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [18]:
SELECT Pessoa.Id, Pessoa.Age, UsoMedicamento.DrugCode, UsoMedicamento.DaysUse
    FROM Pessoa, Medicamento, UsoMedicamento
    WHERE Pessoa.Id = UsoMedicamento.PessoaId
    AND Medicamento.Code = UsoMedicamento.DrugCode
    ORDER BY UsoMedicamento.DaysUse;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [25]:
SELECT DISTINCT Medicamento.Name, UsoMedicamento.DaysUse
    FROM Medicamento, UsoMedicamento
    WHERE Medicamento.Code = UsoMedicamento.DrugCode
    AND DaysUse >= 1
    ORDER BY UsoMedicamento.DaysUse;

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [26]:
SELECT DISTINCT Medicamento.Name, UsoMedicamento.DaysUse
    FROM Medicamento, UsoMedicamento
    WHERE Medicamento.Code = UsoMedicamento.DrugCode
    AND UsoMedicamento.DaysUse > 2000
    ORDER BY UsoMedicamento.DaysUse;